In [1]:
import plotly.figure_factory as ff
import pandas as pd

In [2]:
#calls 
calls = ['https://data.police.uk/api/crimes-street/all-crime?poly=53.933024,%20-1.949214:53.936022,%20-2.049110:54.016624,%20-1.954005:54.009636,%20-2.076643&date=2019-09',
         'https://data.police.uk/api/crimes-street/all-crime?poly=53.933024,%20-1.949214:53.936022,%20-2.049110:54.016624,%20-1.954005:54.009636,%20-2.076643&date=2019-08',
         'https://data.police.uk/api/crimes-street/all-crime?poly=53.933024,%20-1.949214:53.936022,%20-2.049110:54.016624,%20-1.954005:54.009636,%20-2.076643&date=2019-07',
         'https://data.police.uk/api/crimes-street/all-crime?poly=53.933024,%20-1.949214:53.936022,%20-2.049110:54.016624,%20-1.954005:54.009636,%20-2.076643&date=2019-06',
         'https://data.police.uk/api/crimes-street/all-crime?poly=53.933024,%20-1.949214:53.936022,%20-2.049110:54.016624,%20-1.954005:54.009636,%20-2.076643&date=2019-05',
         'https://data.police.uk/api/crimes-street/all-crime?poly=53.933024,%20-1.949214:53.936022,%20-2.049110:54.016624,%20-1.954005:54.009636,%20-2.076643&date=2019-04']

# Create an initial empty data frame
df = pd.DataFrame()
# Make the consecutive calls
for i, call in enumerate(calls):
    # Create the new DataFrame from the data you got
    df_new = pd.read_json(call, orient='records')
    # Merge it with the current data frame
    df = pd.concat([df, df_new])
#decrompess the nested data and add it to the table
df["latitude"] = df.latitude = pd.DataFrame(df.location.values.tolist())['latitude'].astype(float)
df["longitude"] = df.longitude = pd.DataFrame(df.location.values.tolist())['longitude'].astype(float)
#df = df.groupby(['month','id','category'])['location'].apply(','.join).reset_index()    

#make it look pretty with less superfluous crap
craven = df[['id', 'category', 'month', 'longitude', 'latitude']]
craven.info()
craven.head(100)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 550 entries, 0 to 89
Data columns (total 5 columns):
id           550 non-null int64
category     550 non-null object
month        550 non-null object
longitude    550 non-null float64
latitude     550 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 25.8+ KB


,id,category,month,longitude,latitude
0,77291174,anti-social-behaviour,2019-09,-2.018207,53.960188
1,77291161,anti-social-behaviour,2019-09,-2.009627,53.960261
2,77290979,anti-social-behaviour,2019-09,-2.019852,53.957356
3,77288132,anti-social-behaviour,2019-09,-2.003210,53.959182
4,77289110,anti-social-behaviour,2019-09,-2.018130,53.959217
...,...,...,...,...,...
13,76770916,anti-social-behaviour,2019-08,-2.016499,53.957878
14,76773703,anti-social-behaviour,2019-08,-2.015053,53.962165
15,76772495,anti-social-behaviour,2019-08,-1.999309,53.960459
16,76772041,anti-social-behaviour,2019-08,-2.002235,53.961762


In [74]:
import plotly.express as px
fig = px.scatter_mapbox(craven, lon="longitude", lat="latitude", color="category", zoom=14)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [49]:
import numpy as np

violent = craven.loc[craven['category']=="violent-crime"]
#need to figure out how to aggregate the group crime types and count whats in each then add it to the dataframe
#reset index is there in order to get the name for the count column
#longitutde and latitude need to be inside the square brackets or it messes up.
grouped = violent.groupby(['longitude','latitude','month']).size().reset_index(name='count')
#grouped['freq'] = violent.groupby('longitude')['latitude'].value_counts().transform('freq')
grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 4 columns):
longitude    129 non-null float64
latitude     129 non-null float64
month        129 non-null object
count        129 non-null int64
dtypes: float64(2), int64(1), object(1)
memory usage: 4.2+ KB


In [72]:
fig2 = px.scatter_mapbox(grouped, lon="longitude", lat="latitude", color="month", 
                         size="count", zoom=14)
fig2.update_layout(mapbox_style="open-street-map")
fig2.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig2.update_layout(
    title="Violent Crimes in Skipton by month",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)

fig2.show()

To do list:

1) Put the data into github and read the CSV from there so that sam can access it
2) make sure the data is fully readable, removing the spaces commas in the thousands. The thousand commas can be removed during the read CSV and the spaces can be removed with a replace function.
3) use plotly to make the map

call the data from an API
Probably can call the location id's from an api as well

In [73]:
fig3 = px.scatter_mapbox(grouped, lon="longitude", lat="latitude", 
                         size="count", zoom=14, animation_frame="month")
fig3.update_layout(mapbox_style="open-street-map")
fig3.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig3.update_layout(
    title="Violent Crimes in Skipton by month",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)

fig3.show()